## Initial summary of event files

**Dataset**: aomic-piop2 [openNeuro ds002790](https://openneuro.org/datasets/ds002790).

This script does a preliminary summary of the contents of the events files.
The summary includes printing out the column names of each event file so
that they can be manually checked for differences.

The script assumes that the data is in BIDS format and that each BIDS events
file of the form `_events.tsv` has a corresponding events file with
suffix `_events_temp.tsv` that was previously dumped from the `EEG.set` files.

In order to compare the events coming from the BIDS events files and those
from the EEG.set files, the script creates dictionaries of `key` to full path
for each type of file.  The `key` is of the form `sub-xxx_run-y` which
uniquely specify each event file in the dataset. If a dataset contains
multiple sessions for each subject, the `key` should include additional
parts of the file name to uniquely specify each subject.

Keys are specified by a `name_indices` tuple which consists of the
pieces of the file name to include. Here pieces are separated by the
underbar character.

For a file name `sub-001_ses-3_task-target_run-01_events.tsv`,
the tuple (0, 2) gives a key of `sub-001_task-target`,
while the tuple (0, 3) gives a key of `sub-001_run-01`.
The use of dictionaries of file names with such keys makes it
easier to associate related files in the BIDS naming structure.

The setup requires the setting of the following variables for your dataset:

| Variable | Purpose |
| -------- | ------- |
| bids_root_path | Full path to root directory of dataset.|
| exclude_dirs | List of directories to exclude when constructing file lists. |
| entities  | Tuple of entity names used to construct a unique keys representing filenames. <br>(See [Dictionaries of filenames](https://hed-examples.readthedocs.io/en/latest/HedInPython.html#dictionaries-of-filenames-anchor) for examples of how to choose the key.)||
| skip_columns  |  List of column names in the `events.tsv` files to skip in the analysis. |

In [1]:
from hed.tools import BidsTabularDictionary, get_file_list

# Variables to set for the specific dataset
#bids_root_path = '/openneuro/ds002790-download'
bids_root_path = 'T:/ds002790-data'
exclude_dirs = ['derivatives', 'models']
entities = ('sub', 'task')
skip_columns = ['onset', 'duration', 'response_time', 'stop_signal_delay']
tasks = ['emomatching', 'restingstate', 'stopsignal', 'workingmemory']
# tasks = ['stopsignal']
name = 'aomic-piop2'
# Construct the event file dictionaries for the BIDS and for EEG.event files
event_files = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_events",
                            exclude_dirs=exclude_dirs)
bids_dict = BidsTabularDictionary(name, event_files, entities=('sub', 'task'))

In [2]:
task_dicts, leftovers = bids_dict.split_by_entity('task')
print(f"Dataset tasks are [{str(task_dicts.keys())}]")
for task, task_dict in task_dicts.items():
    print(f"\nBIDS-style event files for task {task}\n{task_dict}")


if leftovers:
    print(f"\nThese file did not have a task entity\n{leftovers}")

Dataset tasks are [dict_keys(['stopsignal', 'workingmemory', 'emomatching'])]

BIDS-style event files for task stopsignal

BIDS-style event files for task workingmemory

BIDS-style event files for task emomatching


In [3]:
print(f"\nBIDS-style event file columns:")
for task, task_dict in task_dicts.items():
    print(f"\nTask {task} event file columns:")
    for key, file, rowcount, columns in task_dict.iter_files():
        print(f"{key} [{rowcount} events]: {str(columns)}")



BIDS-style event file columns:

Task stopsignal event file columns:
sub-0001_task-stopsignal [100 events]: ['onset', 'duration', 'trial_type', 'stop_signal_delay', 'response_time', 'response_accuracy', 'response_hand', 'sex']
sub-0002_task-stopsignal [100 events]: ['onset', 'duration', 'trial_type', 'stop_signal_delay', 'response_time', 'response_accuracy', 'response_hand', 'sex']
sub-0003_task-stopsignal [100 events]: ['onset', 'duration', 'trial_type', 'stop_signal_delay', 'response_time', 'response_accuracy', 'response_hand', 'sex']
sub-0004_task-stopsignal [100 events]: ['onset', 'duration', 'trial_type', 'stop_signal_delay', 'response_time', 'response_accuracy', 'response_hand', 'sex']
sub-0005_task-stopsignal [100 events]: ['onset', 'duration', 'trial_type', 'stop_signal_delay', 'response_time', 'response_accuracy', 'response_hand', 'sex']
sub-0006_task-stopsignal [100 events]: ['onset', 'duration', 'trial_type', 'stop_signal_delay', 'response_time', 'response_accuracy', 'respon

In [4]:
from hed.tools import TabularSummary

print('\nBIDS events summary counts:')
for task, task_dict in task_dicts.items():
    dicts_all, dicts_sep = TabularSummary.make_combined_dicts(task_dict.file_dict, skip_cols=skip_columns)
    print(f"\nBIDS-style event info for task {task}:\n{dicts_all}")


BIDS events summary counts:

BIDS-style event info for task stopsignal:
Summary for column dictionary :
   Categorical columns (4):
      response_accuracy (4 distinct values):
         correct: 17061
         incorrect: 923
         miss: 246
         n/a: 4370
      response_hand (2 distinct values):
         left: 9395
         right: 13205
      sex (2 distinct values):
         female: 11360
         male: 11240
      trial_type (3 distinct values):
         go: 15119
         succesful_stop: 4370
         unsuccesful_stop: 3111
   Value columns (0):

BIDS-style event info for task workingmemory:
Summary for column dictionary :
   Categorical columns (3):
      response_accuracy (4 distinct values):
         correct: 3665
         incorrect: 3353
         miss: 631
         n/a: 1311
      response_hand (3 distinct values):
         left: 4235
         n/a: 8
         right: 4717
      trial_type (3 distinct values):
         active_change: 3584
         active_nochange: 3584
   